In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sps
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.3, palette='Set2')

%matplotlib inline

### Одновыборочный T-test

Дана одна выборка $X_1, ..., X_n$.

Критерий проверяет гипотезы

$\mathsf{H}_0\colon \mathsf{E} X = a_0$

$\mathsf{H}_1\colon \mathsf{E} X \not= a_0$

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html#scipy.stats.ttest_1samp">`ttest_1samp`</a>`(a, popmean): statistic, pvalue`

* `a` &mdash; выборка
* `popmean` &mdash; равно $a_0$

In [ ]:
sample = sps.norm(loc=0).rvs(size=100)
sps.ttest_1samp(sample, 0), sps.ttest_1samp(sample, 0.5)

### Двухвыборочный T-test &mdash; независимые выборки

Даны две независимые выборки

* $X_1, ..., X_n$,

* $Y_1, ..., Y_m$.

Критерий проверяет для их гипотезы о равенстве среднего:

$\mathsf{H}_0\colon \mathsf{E} X_1 = \mathsf{E} X_2$

$\mathsf{H}_1\colon \mathsf{E} X_1 \not= \mathsf{E} X_2$

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy.stats.ttest_ind">`ttest_ind`</a>`(a, b, equal_var=True): statistic, pvalue`

`a`, `b` &mdash; выборка

`equal_var` &mdash; известно ли равенство дисперсий

In [ ]:
sample_1 = sps.norm(loc=0).rvs(size=100)
sample_2 = sps.norm(loc=1).rvs(size=100)
sps.ttest_ind(sample_1, sample_2)

In [ ]:
sample_1 = sps.norm(loc=0).rvs(size=100)
sample_2 = sps.norm(loc=1, scale=7).rvs(size=100)
sps.ttest_ind(sample_1, sample_2, equal_var=False)

### Двухвыборочный T-test &mdash; связные выборки

Даны две связные выборки

* $X_1, ..., X_n$,

* $Y_1, ..., Y_n$.


Критерий проверяет для их гипотезы о равенстве среднего:

$\mathsf{H}_0\colon \mathsf{E} X_1 = \mathsf{E} X_2$

$\mathsf{H}_1\colon \mathsf{E} X_1 \not= \mathsf{E} X_2$

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html#scipy.stats.ttest_rel">`ttest_rel`</a>`(a, b): statistic, pvalue`

`a`, `b` &mdash; выборка

In [ ]:
sample_1 = sps.norm(loc=0).rvs(size=100)
sample_2 = sample_1 + sps.norm(loc=0, scale=0.5).rvs(size=100)
sps.ttest_rel(sample_1, sample_2)

In [ ]:
sample_1 = sps.norm(loc=0).rvs(size=100)
sample_2 = sample_1 + sps.norm(loc=0.5, scale=0.5).rvs(size=100)
sps.ttest_rel(sample_1, sample_2)

### Пример: ирисы Фишера

Визуализация данных

In [ ]:
df = sns.load_dataset("iris")

g = sns.PairGrid(df, hue='species')
g.map_lower(sns.kdeplot, cmap ="Blues_d")
g.map_upper(plt.scatter)
g.map_diag(sns.kdeplot, lw=3)
plt.legend();

Как выглядят данные

In [ ]:
df.head()

Виды ирисов

In [ ]:
np.unique(df.species)

In [ ]:
sps.ttest_ind(df[df.species == 'setosa'].sepal_length,
              df[df.species == 'versicolor'].sepal_length,
              equal_var=False)

In [ ]:
sps.ttest_ind(df[df.species == 'virginica'].sepal_length,
              df[df.species == 'versicolor'].sepal_length,
              equal_var=False)

In [ ]:
sps.ttest_ind(df[df.species == 'virginica'].sepal_width,
              df[df.species == 'versicolor'].sepal_width,
              equal_var=False)

*Замечание.* Строго говоря, неоходима поправка на множественное тестирование гипотез.

## AA-тесты: валидация критериев

Напишем функцию, реализующую относительный t-test. Функция возвращает значение статистики, доверительный интервал, p-value.

Для подсчёта статистики воспользуемся следующей формулой

$$R = \overline{X} / \overline{Y} - 1$$

где n - размер выборки. Вам здесь поможет функция `np.var`

Для подсчёта `pvalue` найдите вероятность того, что стандартно распределённая случайная велечина имеет значение по модулю болшее чем

$$z = \sqrt{n}\frac{|R|}{\sqrt{\frac{\overline{(X - \overline{X})^2}}{\overline{Y}^2} + \frac{\overline{(Y - \overline{Y})^2}\overline{X}^2}{\overline{Y}^4}}}$$

Вам понадобится функция `sps.norm.sf`

Посчитайте доверительный интервал. Для этого найдите

$$q = \Phi^{-1}(1-\alpha/2)$$

где $\Phi$ - функция распределения стандартного нормального распределения (воспользуйтесь методом `sps.norm.ppf`), $\alpha$ - уровень значимости. Затем от $R$ отложите

$$\frac{q \cdot \sigma_R}{\sqrt{n}}$$

где $\sigma_R$ - знаменатель из формулы для расчёта $z$.

In [ ]:
def relative_ttest(x, y, alpha=0.05):
    '''
    Относительный t-test.

    Аргументы:
    x, y -- выборки одинакового размера
    alpha -- уровень значимости

    Возвращает:
    stat -- статистика критерия
    left_bound, right_bound -- границы дов. интервала
    pvalue
    '''

    n =
    x_mean =
    y_mean =

    stat =
    std =

    z =
    pvalue =

    q =
    left_bound = stat -
    right_bound = stat +

    return stat, left_bound, right_bound, pvalue

In [ ]:
def calculate_real_alpha(n_errors, n_iter):
    '''
    Оценка реального уровня значимости и его дов. интервала.

    Аргументы:
    n_errors -- количество ошибок в эксперименте
    n_iter -- количество экспериментов

    Возвращает:
    real_alpha -- оценка реального уровня значимости
    left_alpha, right_alpha -- границы соотв. дов. интервала
    '''

    real_alpha = n_errors / n_iter
    std = np.sqrt(real_alpha * (1-real_alpha) / n_iter)
    left_alpha = real_alpha - 2 * std
    right_alpha = real_alpha + 2 * std

    return real_alpha, left_alpha, right_alpha

In [ ]:
def draw_interval(
    real_alpha, left_alpha, right_alpha,
    alpha=0.05, new_fig=True
):
    '''
    Отрисовка интервала для реального уровня значимости
    '''

    if new_fig:
        plt.figure(figsize=(7, 3.5))

    plt.hlines(0, 0, 1, color='black', lw=2, alpha=0.6)
    plt.vlines(alpha, -1, 1, color='red', lw=2, linestyle='--', alpha=0.6)
    plt.fill_between([left_alpha, right_alpha], [0.1]*2, [-0.1]*2, color='green', alpha=0.6)
    plt.scatter(real_alpha, 0, s=300, marker='*', color='red')
    plt.xlim((min(alpha, left_alpha)-1e-3, max(alpha, right_alpha)+1e-3))
    plt.title('Доля отвержений')
    plt.ylim((-0.5, 0.5))
    plt.yticks([])

In [ ]:
def AB_test(
    distr1, distr2, n_iter=10000, sample_size=1000, alpha=0.05
):
    '''
    Проведение серии AB-тестов на искусственных выборках.

    Аргументы:
    distr1, distr2 -- распределения для семплирования выборок
    n_iter -- количество итераций
    sample_size -- размер выборок
    alpha -- уровень значимости
    '''

    n_errors = 0  # количество отвержений H_0

    # Проведение экспериментов
    for _ in tqdm(range(n_iter)):
        x = distr1.rvs(size=sample_size)
        y = distr2.rvs(size=sample_size)
        n_errors += relative_ttest(x, y)[3] < alpha

    # Реальный уровень значимости
    real_alpha, left_alpha, right_alpha = calculate_real_alpha(n_errors, n_iter)
    print('{:.4f} +/- {:.4f}'.format(
        real_alpha, (right_alpha - left_alpha)/2
    ))

    plt.figure(figsize=(14, 3.5))

    # График плотности распределений
    with sns.axes_style("darkgrid"):
        plt.subplot(1, 2, 1)
        for d, label in zip([distr1, distr2], ['A', 'B']):
            grid = np.linspace(d.ppf(0.005) - 0.2, d.ppf(0.995) + 0.2, 1000)
            plt.plot(grid, d.pdf(grid), label=label, lw=3)
        plt.legend()
        plt.title('Плотности выборок')

    # График интервала
    with sns.axes_style("whitegrid"):
        plt.subplot(1, 2, 2)
        draw_interval(
            real_alpha, left_alpha, right_alpha,
            alpha=alpha, new_fig=False
        )

    plt.tight_layout()

Проверим корректность критерия методом AA-тестирования на искусственных данных. Рассмотрите разные виды распределений и разный размер выборки. Представим полученные результаты в удобном виде.

In [ ]:
n_iter = 200000  # количество итераций в одном эксперименте
alpha = 0.05  # уровень значимости

#### Экспоненциальное распределение

In [ ]:
AB_test(sps.expon, sps.expon, n_iter=n_iter, sample_size=10, alpha=alpha)

In [ ]:
AB_test(sps.expon, sps.expon, n_iter=n_iter, sample_size=100, alpha=alpha)

In [ ]:
AB_test(sps.expon, sps.expon, n_iter=n_iter, sample_size=1000, alpha=alpha)

In [ ]:
AB_test(sps.expon, sps.expon, n_iter=n_iter, sample_size=10000, alpha=alpha)

#### Гамма-распределение

In [ ]:
AB_test(sps.gamma(a=3), sps.gamma(a=3), n_iter=n_iter, sample_size=10, alpha=alpha)

In [ ]:
AB_test(sps.gamma(a=3), sps.gamma(a=3), n_iter=n_iter, sample_size=100, alpha=alpha)

In [ ]:
AB_test(sps.gamma(a=3), sps.gamma(a=3), n_iter=n_iter, sample_size=10000, alpha=alpha)

#### Распределение Коши

In [ ]:
AB_test(sps.cauchy(loc=1000), sps.cauchy(loc=1000), n_iter=n_iter, sample_size=1000, alpha=alpha)

#### Нормальное распределение

In [ ]:
AB_test(sps.norm, sps.norm, n_iter=n_iter, sample_size=100, alpha=alpha)

In [ ]:
AB_test(sps.norm, sps.norm, n_iter=n_iter, sample_size=1000, alpha=alpha)

**Вывод:**

#### Мощность для распределения

Зафиксируйте тип распределения и размер выборки. Исследуйте мощность критерия помощью добавления смещения к одной из выборок.

In [ ]:
alpha=0.05

In [ ]:
AB_test(sps.gamma(a=3), sps.gamma(a=3, loc=0.01), n_iter=n_iter, sample_size=100, alpha=alpha)

In [ ]:
AB_test(sps.gamma(a=3), sps.gamma(a=3, loc=0.1), n_iter=n_iter, sample_size=100, alpha=alpha)

In [ ]:
AB_test(sps.gamma(a=3), sps.gamma(a=3, loc=1), n_iter=n_iter, sample_size=100, alpha=alpha)

Рассмотрим сетку значений сдвига

In [ ]:
n_errors = []  # количество отвержений H_0
sample_size = 1000
shifts = np.linspace(0, 0.5, 21)

for loc in tqdm(shifts):
    n_errors.append(0)
    for _ in range(n_iter):
        x = sps.gamma.rvs(a=3, size=sample_size)
        y = sps.gamma.rvs(a=3, size=sample_size) + loc
        n_errors[-1] += relative_ttest(x, y)[3] < alpha

In [ ]:
plt.plot(shifts, np.array(n_errors)/n_iter, lw=3)
plt.title('Мощность относительного t-test')
plt.xlabel('Смещение')
plt.ylabel('Мощность');

**Вывод:**